<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w4_model_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training & Evaluation

## ##Training & Evaluation is currently being carried out in google Colaboratory due to less free resources available in IBM watson studio

# Download Data

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
'''
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
'''

In [1]:
#download_file_cos(credentials_2,'train_feat_eng.csv','train_feat_eng.csv')
#ownload_file_cos(credentials_2,'test_feat_eng.csv','test_feat_eng.csv')
#download_file_cos(credentials_2,'sample_submission.csv','sample_submission.csv')
!ls

dipole_moments.csv.zip		       structures.csv.zip
kaggle.json			       structures.zip
magnetic_shielding_tensors.csv.zip     test.csv.zip
mulliken_charges.csv.zip	       test_etl.csv
potential_energy.csv.zip	       test_feat_eng_v2.csv
sample_data			       train.csv.zip
sample_submission.csv.zip	       train_etl.csv
scalar_coupling_contributions.csv.zip  train_feat_eng_v2.csv


# Creating training, validation and testing data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
%matplotlib inline

In [2]:
tf.__version__

'1.14.0'

In [3]:
train=pd.read_csv('train_feat_eng_v2.csv')
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,-0.535689,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133922,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133922,-0.535689,84.8074


In [4]:
train.shape

(4658147, 37)

In [27]:
test=pd.read_csv('test_feat_eng_v2.csv')
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_0_enc,atom_1_enc,scalar_coupling_constant,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,0.207019
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019


In [28]:
test.drop('scalar_coupling_constant',axis=1,inplace=True)
test=test[['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'distance', 'type_enc', 'atom_1_enc', 'atom_0_enc', 'rc_A',
       'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve',
       'U0', 'U', 'H', 'G', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
       'linear', 'mulliken_min', 'mulliken_max', 'mulliken_mean',
       'mulliken_atom_0', 'mulliken_atom_1']]
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,0,1,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,0,1,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,0.207019
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,0,1,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,0,1,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019


In [29]:
test.shape

(2505542, 36)

In [8]:
train.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'distance', 'type_enc', 'atom_1_enc', 'atom_0_enc', 'rc_A',
       'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve',
       'U0', 'U', 'H', 'G', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
       'linear', 'mulliken_min', 'mulliken_max', 'mulliken_mean',
       'mulliken_atom_0', 'mulliken_atom_1', 'scalar_coupling_constant'],
      dtype='object')

In [9]:
test.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'distance', 'type_enc', 'atom_1_enc', 'atom_0_enc', 'rc_A',
       'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve',
       'U0', 'U', 'H', 'G', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
       'linear', 'mulliken_min', 'mulliken_max', 'mulliken_mean',
       'mulliken_atom_0', 'mulliken_atom_1'],
      dtype='object')

In [10]:
x=train.drop('scalar_coupling_constant',axis=1)
x.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,-0.535689
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133922
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133922,-0.535689


In [11]:
y=train['scalar_coupling_constant']
y.head()

0    84.8076
1   -11.2570
2   -11.2548
3   -11.2543
4    84.8074
Name: scalar_coupling_constant, dtype: float64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3)

In [17]:
print('x_train: {}   y_train: {}'.format(x_train.shape,y_train.shape))
print('x_val: {}   y_val: {}'.format(x_val.shape,y_val.shape))

x_train: (3260702, 36)   y_train: (3260702,)
x_val: (1397445, 36)   y_val: (1397445,)


In [18]:
x_test=test
x_test.shape

(2505542, 36)

# Training & Evaluation

In [19]:
input_shape=(36,)
model_nn=keras.Sequential()
model_nn.add(keras.layers.Dense(64,activation='relu',input_shape=input_shape))
model_nn.add(keras.layers.Dense(128,activation='relu'))
#model_nn.add(keras.layers.Dense(256,activation='relu'))
#model_nn.add(keras.layers.Dense(512,activation='relu'))
model_nn.add(keras.layers.Dense(128,activation='relu'))
model_nn.add(keras.layers.Dense(64,activation='relu'))
model_nn.add(keras.layers.Dense(8,activation='relu'))
model_nn.add(keras.layers.Dense(1))

model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                2368      
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 520       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 9         
Total params: 35,985
Trainable params: 35,985
Non-trainable params: 0
__________________________________________________

In [0]:
model_nn.compile(optimizer='adam',loss='mean_absolute_error')

In [23]:
model_nn.fit(x_train,y_train,epochs=5,batch_size=512,validation_data=(x_val,y_val))

Train on 3260702 samples, validate on 1397445 samples
Epoch 1/5
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.4638 - val_loss: 3.3371
Epoch 2/5
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.4044 - val_loss: 3.2796
Epoch 3/5
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.3791 - val_loss: 3.3623
Epoch 4/5
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.3514 - val_loss: 3.3446
Epoch 5/5
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.3317 - val_loss: 3.2794


# Train on whole data

In [24]:
model_nn.fit(x,y,epochs=5,batch_size=512,validation_data=(x_val,y_val))

Train on 4658147 samples, validate on 1397445 samples
Epoch 1/5
4658147/4658147 [==============================] - 42s 9us/sample - loss: 3.3060 - val_loss: 3.3226
Epoch 2/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.2830 - val_loss: 3.2570
Epoch 3/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.2622 - val_loss: 3.2951
Epoch 4/5
4658147/4658147 [==============================] - 42s 9us/sample - loss: 3.2442 - val_loss: 3.2489
Epoch 5/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.2228 - val_loss: 3.3035


# Serialize the model

In [0]:
tf.keras.models.save_model(model_nn,'kaggle_model_nn_v2_04.h5',overwrite=True,include_optimizer=True)

In [0]:
!ls

# Create Submission file

In [37]:
import gc
gc.collect()

634

In [30]:
#test=pd.read_csv('test_feat_eng_v2.csv')
test_predictions=model_nn.predict(test,verbose=1,batch_size=10000)
test_predictions[:10]

2505542/2505542 [==============================] - 1s 1us/sample


array([[107.41792  ],
       [176.21368  ],
       [ -8.262182 ],
       [179.207    ],
       [106.91041  ],
       [ 89.81072  ],
       [ -5.2951846],
       [ -9.833796 ],
       [ -9.950052 ],
       [ 88.95819  ]], dtype=float32)

In [31]:
test_predictions.shape

(2505542, 1)

In [32]:
sample_submission=pd.read_csv('sample_submission.csv.zip')
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [33]:
sample_submission['scalar_coupling_constant']=test_predictions
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,107.417923
1,4658148,176.213684
2,4658149,-8.262182
3,4658150,179.207001
4,4658151,106.910408


In [0]:
sample_submission.to_csv('sample_submission_external_feat_v2_01.csv',index=False)

In [35]:
!ls

data.covs.pickle
dipole_moments.csv.zip
kaggle.json
kaggle_model_nn_v2_04.h5
magnetic_shielding_tensors.csv.zip
mulliken_charges.csv.zip
potential_energy.csv.zip
sample_data
sample_submission.csv.zip
sample_submission_distance_baseline_test_02.csv
sample_submission_external_feat_v2_01.csv
scalar_coupling_contributions.csv.zip
structures.csv.zip
structures.zip
test.csv.zip
test_etl.csv
test_feat_eng_v2.csv
train.csv.zip
train_etl.csv
train_feat_eng_v2.csv


In [44]:
!head sample_submission_distance_baseline_test.csv

id,scalar_coupling_constant
4658147,-3.2472324
4658148,91.86721
4658149,10.933329
4658150,107.796974
4658151,1.5662003
4658152,107.1634
4658153,2.08945
4658154,-11.683802
4658155,-10.687826


# Upload to IBM object store

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [0]:
upload_file_cos(credentials_2,'sample_submission_test_3.csv','sample_submission_test_3.csv')

In [0]:
upload_file_cos(credentials_2,'kaggle_model_nn_03.h5','kaggle_model_nn_03.h5')